In [2]:
# install pretrained BERT model
!pip3 install pytorch_pretrained_bert

    100% |████████████████████████████████| 122kB 3.8MB/s 


In [57]:
'''
  code by Minho Ryu @bzantium
  
'''

# Sentiment Analysis

import torch.nn as nn
import torch.optim as optim
import numpy as np
from pytorch_pretrained_bert import BertModel, BertTokenizer, BertAdam
from copy import deepcopy

from torch import LongTensor as LT

# words sentences
sentences = ["i love you", "he loves me", "she likes baseball", "i hate you", "sorry for that", "this is awful"]
targets = [1, 1, 1, 0, 0, 0]  # 1 is good, 0 is bad.

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

inputs = []
for sent in sentences:
    tokens =  ['[CLS]'] + tokenizer.tokenize(sent) + ['[SEP]']
    sequence = tokenizer.convert_tokens_to_ids(tokens)
    inputs.append(sequence)

input_batch = LT(inputs)
target_batch = LT(targets)

input_dims = 768
output_dims = 2

BERT_model = BertModel.from_pretrained('bert-base-uncased')


class BERTEncoder(nn.Module):
    def __init__(self):
        super(BERTEncoder, self).__init__()
        self.encoder = deepcopy(BERT_model)

    def forward(self, x, token_id=None, mask=None):
        _, feat = self.encoder(x, token_type_ids=token_id, attention_mask=mask, output_all_encoded_layers=False)
        return feat


class BERTClassifier(nn.Module):
    def __init__(self, input_dims, output_dims):
        super(BERTClassifier, self).__init__()
        self.dropout = nn.Dropout(p = 0.1)
        self.classifier = nn.Linear(input_dims, output_dims)
    def forward(self, x):
        x = self.dropout(x)
        out = self.classifier(x)
        return out

class BERTForSequenceClassifier(nn.Module):
    def __init__(self, input_dims, output_dims):
        super(BERTForSequenceClassifier, self).__init__()
        self.encoder = BERTEncoder()
        self.classifier = BERTClassifier(input_dims, output_dims)
    
    def forward(self, x):
        x = self.encoder(x)
        out = self.classifier(x)
        return out
        
        
model = BERTForSequenceClassifier(input_dims, output_dims)
criterion = nn.CrossEntropyLoss()
optimizer = BertAdam(model.parameters(), lr=5e-5)

# Training
model.train()
for epoch in range(15):
    optimizer.zero_grad()
    output = model(input_batch)

    # output : [batch_size, num_classes], target_batch : [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, target_batch)
    if (epoch + 1) % 3 == 0:
        print('Epoch:', '%02d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()
    
# Test
test_text = ['sorry hate you', 'you love me']
tests = []
for sent in test_text:
    tokens =  ['[CLS]'] + tokenizer.tokenize(sent) + ['[SEP]']
    sequence = tokenizer.convert_tokens_to_ids(tokens)
    tests.append(sequence)

test_batch = LT(tests)

# Predict
model.eval()
result = model(test_batch).data.max(1)[1]
for i, text in enumerate(test_text):
  if result[i] == 1:
      print("\'" + text + "\'", "is good :)")
  else:
      print("\'"+ text + "\'", "is bad :(")

Epoch: 03 cost = 0.336113
Epoch: 06 cost = 0.076280
Epoch: 09 cost = 0.005214
Epoch: 12 cost = 0.000949
Epoch: 15 cost = 0.000777
'sorry hate you' is bad :(
'you love me' is good :)


In [59]:
# Next Sentence Prediction

import torch.nn as nn
import torch.optim as optim
import numpy as np
from termcolor import colored
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam

from torch import LongTensor as LT


# words sentences
next_sentences = ["Once upon a time, there was a kind girl named Cinderella.",
                  "All of the animals loved her, especially two mice named Gus and Jaq.", 
                  "They'd do anything for the girl they called Cinderelly.",
                  "Cinderella lived with her stepmother and her two stepsisters, Anastasia and Drizella.",
                  "They were very mean to Cinderella, making her work all day cleaning, sewing, and cooking.",
                  "She tried her best to make them happy.",
                  "Cinderella's stepmother, Lady Tremaine, was cold, cruel, and jealous of Cinderella’s charm and beauty."]

not_next_sentences = ["One day, a messenger arrived with a special invitation.",
                      "Lady Tremaine didn't want Cinderella to go to the ball.",
                      "She wanted the Prince to meet Anastasia and Drizella.",
                      "It was a bit old-fashioned, but Cinderella could make it beautiful!",
                      "Cinderella was overjoyed when she saw the dress.",
                      "They ripped the dress and pulled off the beads.",
                      "Suddenly, her fairy godmother appeared.",
                      "Lady Tremaine didn't stop them.",
                      "Cinderella's dream of going to the ball was through.",
                      "At the ball, Prince Charming couldn't take his eyes off Cinderella.",
                      "The orchestra played, and the Prince began to dance with the wonderful girl whose name he still didn't know.",
                      "When the clock struck midnight, the magic spell would wear off!"]

targets = [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]  # 1 is Next, 0 is Not Next.

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

inputs = []
attn_masks = []
segment_ids = []
maxlen = 50

for i in range(len(next_sentences) - 1):
    token_a = ['[CLS]'] + tokenizer.tokenize(next_sentences[i]) + ['[SEP]']
    token_b = tokenizer.tokenize(next_sentences[i+1]) + ['[SEP]']
    tokens =  token_a + token_b
    sequence = tokenizer.convert_tokens_to_ids(tokens)
    seqlen = len(sequence)
    padding = [0] * (maxlen - seqlen)
    sequence += padding
    attn_mask = [1] * seqlen + padding
    segment_id = [0] * len(token_a) + [1] * len(token_b) + padding
    inputs.append(sequence)
    attn_masks.append(attn_mask)
    segment_ids.append(segment_id)

for i in range(0, len(not_next_sentences) - 1, 2):
    token_a = ['[CLS]'] + tokenizer.tokenize(not_next_sentences[i]) + ['[SEP]']
    token_b = tokenizer.tokenize(not_next_sentences[i+1]) + ['[SEP]']
    tokens =  token_a + token_b
    sequence = tokenizer.convert_tokens_to_ids(tokens)
    seqlen = len(sequence)
    padding = [0] * (maxlen - seqlen)
    sequence += padding
    attn_mask = [1] * seqlen + padding
    segment_id = [0] * len(token_a) + [1] * len(token_b) + padding
    inputs.append(sequence)
    attn_masks.append(attn_mask)
    segment_ids.append(segment_id)
    
input_batch = LT(inputs)
target_batch = LT(targets)
attn_masks = LT(attn_masks)
segment_ids = LT(segment_ids)

input_dims = 768
output_dims = 2


model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = output_dims)
criterion = nn.CrossEntropyLoss()
optimizer = BertAdam(model.parameters(), lr=5e-5)

# Training
model.train()
for epoch in range(15):
    optimizer.zero_grad()
    loss = model(input_batch, segment_ids, attn_masks, target_batch)
    if (epoch + 1) % 3 == 0:
        print('Epoch:', '%02d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()
    
# Test
test_text = ["Once upon a time, there was a kind girl named Cinderella.",
             "All of the animals loved her, especially two mice named Gus and Jaq.",
             "When the clock struck midnight, the magic spell would wear off!"]

tests = []
test_attn_masks = []
test_segment_ids = []

for i in range(len(test_text) - 1):
    token_a = ['[CLS]'] + tokenizer.tokenize(test_text[i]) + ['[SEP]']
    token_b = tokenizer.tokenize(test_text[i+1]) + ['[SEP]']
    tokens =  token_a + token_b
    sequence = tokenizer.convert_tokens_to_ids(tokens)
    seqlen = len(sequence)
    padding = [0] * (maxlen - seqlen)
    sequence += padding
    attn_mask = [1] * seqlen + padding
    segment_id = [0] * len(token_a) + [1] * len(token_b) + padding
    tests.append(sequence)
    test_attn_masks.append(attn_mask)
    test_segment_ids.append(segment_id)

test_batch = LT(tests)
test_segment_ids = LT(test_segment_ids)
test_attn_masks = LT(test_attn_masks)

# Predict
model.eval()
result = model(test_batch, test_segment_ids, test_attn_masks).data.max(1)[1]
for i in range(len(test_text) - 1):
  if result[i] == 1:
      print("\'" + test_text[i+1] + "\' is", colored('next sentence', 'blue'), "of \'" + test_text[i] + "\'")
  else:
      print("\'" + test_text[i+1] + "\' is", colored('not next sentence', 'red'), "of \'" + test_text[i] + "\'")

Epoch: 03 cost = 0.382100
Epoch: 06 cost = 0.068060
Epoch: 09 cost = 0.005214
Epoch: 12 cost = 0.000469
Epoch: 15 cost = 0.000347
'All of the animals loved her, especially two mice named Gus and Jaq.' is next sentence of 'Once upon a time, there was a kind girl named Cinderella.'
'When the clock struck midnight, the magic spell would wear off!' is not next sentence of 'All of the animals loved her, especially two mice named Gus and Jaq.'


In [55]:
# Question Answering

import torch.nn as nn
import torch.optim as optim
import numpy as np
from termcolor import colored
from pytorch_pretrained_bert import BertTokenizer, BertForQuestionAnswering, BertAdam

from torch import LongTensor as LT

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# example sentences
context_sentences = ['I had a meal at the indian restaurant today.',
                     'he had a wedding ceremony yesterday.',
                     'she will go to a graduation ceremony tomorrow.']

question_sentences = ['where did i have a meal today?',
                      'what did he do yesterday?',
                      'where will she go tomorrow?']

input_ids = []
input_mask = []
segment_ids = []
maxlen = 80

for context, question in zip(context_sentences, question_sentences):
  tokens =  ['[CLS]'] + tokenizer.tokenize(question) + ['[SEP]'] + tokenizer.tokenize(context) + ['[SEP]']
  sequence = tokenizer.convert_tokens_to_ids(tokens)
  padding = [0] * (maxlen - len(tokens))
  attn_mask = [1] * len(sequence) + padding
  sequence += padding
  segment_id = [0] * len(sequence)
  input_ids.append(sequence)
  input_mask.append(attn_mask)
  segment_ids.append(segment_id)
  
start_positions = [16, 11, 13]
end_positions = [18, 13, 15]

input_ids = LT(input_ids)
input_mask = LT(input_mask)
segment_ids = LT(segment_ids)
start_positions = LT(start_positions)
end_positions = LT(end_positions)

model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
criterion = nn.CrossEntropyLoss()
optimizer = BertAdam(model.parameters(), lr=5e-5)

# Training
model.train()
for epoch in range(15):
    optimizer.zero_grad()
    loss = model(input_ids, segment_ids, input_mask, start_positions, end_positions)

    if (epoch + 1) % 3 == 0:
        print('Epoch:', '%02d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()
    
def tokens_to_sentence(input_id, start_position, end_position):
    tokens = tokenizer.convert_ids_to_tokens(input_id[start_position:end_position])
    sentence = ''
    for token in tokens:
        if token[:2] == '##':
            sentence += token[2:]
        elif len(sentence) == 0:
            sentence += token
        else:
          sentence += ' ' + token
          
    return sentence

# Prediction
model.eval()
(start_predictions, end_predictions) = model(input_ids, segment_ids, input_mask)
start_predictions, end_predictions = start_predictions.data.max(1)[1], end_predictions.data.max(1)[1]

print('\n[train predictions]')
for i, (cs, qs) in enumerate(zip(context_sentences, question_sentences)):
    print('Context:', cs)
    print('Question:', qs)
    sentence = tokens_to_sentence(input_ids[i].numpy(), start_predictions[i].numpy(), end_predictions[i].numpy())
    print('Answer:', sentence)

Epoch: 03 cost = 0.895173
Epoch: 06 cost = 0.026652
Epoch: 09 cost = 0.005986
Epoch: 12 cost = 0.752293
Epoch: 15 cost = 0.025888

[train predictions]
Context: I had a meal at the indian restaurant today.
Question: where did i have a meal today?
Answer: indian restaurant
Context: he had a wedding ceremony yesterday.
Question: what did he do yesterday?
Answer: wedding ceremony
Context: she will go to a graduation ceremony tomorrow.
Question: where will she go tomorrow?
Answer: graduation ceremony
